In [1]:
# !pip install recipe-scrapers
# !pip install yagmail

In [12]:
debug_level = 0

In [13]:
from recipe_scrapers import scrape_me
from datetime import datetime
import os

In [14]:
def debug1(debug_string):
    if debug_level >= 1:
        print('\nDebug1:\n' + debug_string)
    return

def debug2(debug_string):
    if debug_level >= 2:
        print('\nDebug2:\n' + debug_string)
    return

def debug3(debug_string):
    if debug_level >= 3:
        print('\nDebug3:\n' + debug_string)
    return

def debug4(debug_string):
    if debug_level >= 4:
        print('\nDebug4:\n' + debug_string)
    return

In [15]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
password = os.environ.get("MYPASS")
debug4(password)

In [16]:
# import getpass

# password = getpass.getpass(prompt = 'Enter the password')

In [17]:

# datetime object containing current date and time
right_now = datetime.now()

# dd/mm/YY H:M:S
dt_string = right_now.strftime("%d/%m/%Y %H:%M:%S")
debug4('Date and Time: ' + dt_string)

In [18]:
def markup_recipe(rTitle,rURL,rImage,rIngredients,rInstructions):

    now = datetime.now()
    rDate = now.strftime("%m/%d/%Y %H:%M")
    
    recipe_markup = 'Title: ' + rTitle + '<br>\nDate: ' + rDate + '\nCategory: Recipes<br>\n'
    recipe_markup = recipe_markup + '__[' + rTitle + ']('+ rURL + ')__<br>\n'
    recipe_markup = recipe_markup + """<img src='""" + rImage + """'><br>\n"""
    rIngredients = rIngredients.replace('\n','<br>\n')
    recipe_markup = recipe_markup + '> ' + rIngredients + '<br>\n\n'
    rInstructions = rInstructions.replace('\n', '<br>\n\n1. ')
    recipe_markup = recipe_markup + '1. ' + rInstructions
    debug4(recipe_markup)
    return(recipe_markup)
   
    

In [19]:
# Different ways to scrape during testing Comment out all but one

# scraper=scrape_me('https://cooking.nytimes.com/recipes/1016595-hamburgers-diner-style')
# scraper=scrape_me('https://cooking.nytimes.com/recipes/1022421-bulgogi-eggplant')
# scraper=scrape_me('https://www.seriouseats.com/fish-fragrant-eggplants-sichuan-braised-eggplant-with-garlic-ginger-and-chilies')
# scraper = scrape_me('https://www.seriouseats.com/grilled-shrimp-rolls-5195448?utm_campaign=seriouseats&utm_medium=email&utm_source=cn_nl&utm_content=24791524.124531&utm_term=')

scraper = scrape_me(input('Recipe URL:'))

Recipe URL: https://cooking.nytimes.com/recipes/1016595-hamburgers-diner-style


In [20]:
# Consolidate the recipe elements

recipe_ingredients =scraper.ingredients()

# Add a newline after each ingredient and convert the list to a string
recipe_ing_temp = [sub + '\n' for sub in recipe_ingredients]
recipe_ingredients = ' '.join(recipe_ing_temp)

recipe_title = scraper.title()
recipe_instructions = scraper.instructions()
recipe_image = scraper.image()
recipe_URL = scraper.url
recipe_site_name = scraper.site_name()

formatted_recipe = markup_recipe(recipe_title, recipe_URL, recipe_image, recipe_ingredients, recipe_instructions)
debug3(formatted_recipe)

In [21]:


# Change the current working directory to the Website content directory
debug4("Original working directory: " + os.getcwd())
os.chdir('./Recipes_Website/content')
debug4("Website content directory: " + os.getcwd())

with open(recipe_title + '.md', 'w',encoding="utf-8") as writer:
    writer.write(formatted_recipe)
    writer.close()
    
# Restore the CWD to the original
os.chdir('../../')
debug4("Restored to Original working directory: " + os.getcwd())


In [22]:
# Prettify the message payload

message = recipe_title + '\n\n' + recipe_URL + '\n\n '

recipe_instructions = recipe_instructions.replace('\n','\n\n')

# Assemble the full message
message = message + recipe_ingredients + '\n' + recipe_instructions

# Is gmail filtering messages with offensive phrases? 
# if you change Asian to something different, it works.
# See the yagmail example below for a different substitution

message = message.replace('Asian', 'slender')

# email setup and code from tutorial at https://realpython.com/python-send-email/

sender_email = "shapim919@gmail.com"  # Enter your address
receiver_email = "iironmiike1@gmail.com"  # Enter receiver address

  # Simplified way  to send the message (from the same tutorial as above)

import yagmail
yag = yagmail.SMTP(sender_email, password)

# remove 'Asian eggplant' from the recipe ingredients and it works. Go figure
# (this is when using the Eggplant Bulgogi recipe)

ings = recipe_ingredients.replace('Asian eggplant','slender eggplant')

mixed_contents = [recipe_title, '\n', recipe_URL,'\n',recipe_image, '\n', ings, '\n', recipe_instructions]

debug3(" Email payload for yagmail:\n" + '\n'.join(str(items) for items in mixed_contents))

yag.send(receiver_email, recipe_title, mixed_contents)

{}